In [139]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q htt  https://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (122 kB/s)

In [179]:
df=spark.read.format("csv").option("header","true").option("inferschema","true").load("/content/Josna.csv");
df.show()

+---+------+---------+----+-------+
| ID|  City|   Degree| Age|Married|
+---+------+---------+----+-------+
|  1|Lisbon|     null|  25|      0|
|  2|Berlin| Bachelor|  25|      1|
|  3|Lisbon|     null|  30|      1|
|  4|Lisbon| Bachelor|  30|      1|
|  5|Berlin| Bachelor|  18|      1|
|  6|Lisbon| Bachelor|null|      0|
|  7|Berlin|  Masters|  30|      1|
|  8|Berlin|No Degree|null|      0|
|  9|Berlin|  Masters|  25|      1|
| 10|Madrid|  Masters|  25|      1|
+---+------+---------+----+-------+



In [154]:
df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- City: string (nullable = true)
 |-- Degree: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Married: integer (nullable = true)



# **Get the Number of Null values for Individual Columns**

In [155]:
ms=df.columns;
gs=[];
for i in ms:
  gs.append(df.filter(df[i].isNull()).count());

In [156]:
gs

[0, 0, 2, 2, 0]

In [157]:
from pyspark.sql.functions import *

# **Separate all the Number Type & String type columns**

In [158]:
ms=df.columns;
input_columns_Number_Type=[];
input_columns_String_Type=[];
for i in ms:
  if str(df.schema[i].dataType) in ('IntegerType','DoubleType'):
    input_columns_Number_Type.append(i);
  elif str(df.schema[i].dataType)=='StringType':
    input_columns_String_Type.append(i);

In [159]:
print(input_columns_Number_Type)
print(input_columns_String_Type)

['ID', 'Age', 'Married']
['City', 'Degree']


# **Automatic Mean Imputation**

In [160]:
def fill_using_mean(dfa,cola):
  for c in dfa.columns:
    if c in cola:
      ms=dfa.select(mean(c).alias("Ammao")).collect()[0].asDict()['Ammao'];
      dfa=dfa.na.fill({c:ms});
  return dfa;

In [161]:
dfa=fill_using_mean(df,input_columns_Number_Type)

In [162]:
dfa.show()

+---+------+---------+---+-------+
| ID|  City|   Degree|Age|Married|
+---+------+---------+---+-------+
|  1|Lisbon|     null| 25|      0|
|  2|Berlin| Bachelor| 25|      1|
|  3|Lisbon|     null| 30|      1|
|  4|Lisbon| Bachelor| 30|      1|
|  5|Berlin| Bachelor| 18|      1|
|  6|Lisbon| Bachelor| 26|      0|
|  7|Berlin|  Masters| 30|      1|
|  8|Berlin|No Degree| 26|      0|
|  9|Berlin|  Masters| 25|      1|
| 10|Madrid|  Masters| 25|      1|
+---+------+---------+---+-------+



# **Automatic Mode Imputation**

In [163]:
def fill_using_mode(dfa,cola):
  for c in dfa.columns:
    if c in cola:
      msao=dfa.groupBy(c).agg(count(c).alias('ms')).orderBy('ms',ascending=False).collect()[0].asDict()[c];
      dfa=dfa.na.fill({c:msao});
  return dfa;

In [164]:
dfa=fill_using_mode(dfa,input_columns_String_Type)

In [165]:
dfa.show()

+---+------+---------+---+-------+
| ID|  City|   Degree|Age|Married|
+---+------+---------+---+-------+
|  1|Lisbon| Bachelor| 25|      0|
|  2|Berlin| Bachelor| 25|      1|
|  3|Lisbon| Bachelor| 30|      1|
|  4|Lisbon| Bachelor| 30|      1|
|  5|Berlin| Bachelor| 18|      1|
|  6|Lisbon| Bachelor| 26|      0|
|  7|Berlin|  Masters| 30|      1|
|  8|Berlin|No Degree| 26|      0|
|  9|Berlin|  Masters| 25|      1|
| 10|Madrid|  Masters| 25|      1|
+---+------+---------+---+-------+

